In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_System_DISC.pq')


# days_since_creation


In [ ]:
# The number of days since the machine was created.
# This feature can be created from the Creation_Date0 field from system_DISC.

In [ ]:
days_since_creation = df.groupby(['ItemKey', 'RWB_EFFECTIVE_DATE'])['Creation_Date0'].agg('max').reset_index()

In [ ]:
days_since_creation['RWB_EFFECTIVE_DATE'] = pd.to_datetime(days_since_creation['RWB_EFFECTIVE_DATE']).dt.date
days_since_creation['Creation_Date0'] = pd.to_datetime(days_since_creation['Creation_Date0'])

In [ ]:
days_since_creation['Creation_Date0_Days'] = days_since_creation['Creation_Date0'].apply(lambda x: x.date())
days_since_creation['Days Since Creation'] = days_since_creation['RWB_EFFECTIVE_DATE'] - days_since_creation['Creation_Date0_Days']

In [ ]:
# days_since_creation.drop(columns='Creation_Date0', inplace=True)
days_since_creation['Days Since Creation'] = days_since_creation['Days Since Creation'].apply(lambda x: x.days)
days_since_creation

In [ ]:
# is_virtual_machine
# Identifies if the machine is a virtual machine or not.
# This can be pulled directly from the Is_Virtual_Macihine0.

# is_client
# Identifies if the machine is a SCCM client or not.
# This can be pulled directly from the Client0 attribute in Persist.System_DISC.

# is_active
# Identifies if the machine is active or not.
# This can be pulled directly from the Active0 attribute in Persist.System_DISC.

In [ ]:
features = df.groupby(['ItemKey', 'RWB_EFFECTIVE_DATE']).agg(
    {
        'Is_Virtual_Machine0':'max',
        'Client0':'max',
        'Active0':'max'
    }
).reset_index()
features


# last_logon_date_age


In [ ]:
# Number of days since a user has logged onto the machine.
# Calculate using the Last_Logon_Timestamp0 attribute from Persist.System_DISC
df['Last_Logon_Timestamp0'] = pd.to_datetime(df['Last_Logon_Timestamp0'])
days_since_last_logon = df.groupby(['ItemKey', 'RWB_EFFECTIVE_DATE'])['Last_Logon_Timestamp0'].agg('max').reset_index()

In [ ]:
days_since_last_logon['RWB_EFFECTIVE_DATE'] = pd.to_datetime(days_since_last_logon['RWB_EFFECTIVE_DATE']).dt.date
days_since_last_logon['Last_Logon_Timestamp0_Date'] = days_since_last_logon['Last_Logon_Timestamp0'].apply(lambda x: x.date())

In [ ]:
days_since_last_logon['Days Since Last Logon'] = days_since_last_logon['RWB_EFFECTIVE_DATE'] - days_since_last_logon['Last_Logon_Timestamp0_Date']
days_since_last_logon['Days Since Last Logon'] = days_since_last_logon['Days Since Last Logon'].apply(lambda x: x.days)
days_since_last_logon

In [ ]:
days_since_creation['RWB_EFFECTIVE_DATE'] = days_since_creation['RWB_EFFECTIVE_DATE'].astype('str')
days_since_last_logon['RWB_EFFECTIVE_DATE'] = days_since_last_logon['RWB_EFFECTIVE_DATE'].astype('str')


# Join them together.


In [ ]:
combined = features.merge(days_since_creation[['ItemKey', 'RWB_EFFECTIVE_DATE', 'Days Since Creation']], on=['ItemKey', 'RWB_EFFECTIVE_DATE'])
combined = combined.merge(days_since_last_logon[['ItemKey', 'RWB_EFFECTIVE_DATE', 'Days Since Last Logon']], on=['ItemKey', 'RWB_EFFECTIVE_DATE'])
combined

In [ ]:
combined.to_parquet('../do_not_commit/FeatureDatasets/system_disc_features.pq')